In [2]:
!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!java -version

!pip install pyspark


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [162]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer,VectorAssembler,OneHotEncoder
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Question 1:Find the cars with the worst fuel efficiency (lowest mpg) for each origin. 

In [9]:
spark = SparkSession.builder.appName("Spark Dataframe Clustering").getOrCreate()
mpgDF = spark.read\
          .option("header","true")\
          .option("inferSchema", "true")\
          .option("delimiter",",")\
          .csv("auto-mpg.data.txt")

In [11]:
mpgDF.show(2)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 2 rows



In [18]:
mpgDF.createOrReplaceTempView("answer_1mpg")

res_answer_one_df = spark.sql("""select sum(nvl(mpg,0)),
case 
when origin = 1 then 'USA'
when origin = 2 then 'Europe'
when origin = 3 then 'Asia' end as origin from answer_1mpg group by origin order by sum(mpg) asc""").show(100)

+------------------+------+
|  sum(nvl(mpg, 0))|origin|
+------------------+------+
|            1952.4|Europe|
|2405.5999999999995|  Asia|
| 5000.799999999998|   USA|
+------------------+------+



Question 2 Add a new column named “Car-Type” that has following values according to
acceleration.

In [19]:
spark = SparkSession.builder.appName("Spark Dataframe Clustering").getOrCreate()
mpgquestion2 = spark.read\
          .option("header","true")\
          .option("inferSchema", "true")\
          .option("delimiter",",")\
          .csv("auto-mpg.data.txt")

In [32]:
mpgDF.createOrReplaceTempView("answer_2mpg")

res_answer_two_df = spark.sql("""select 
case 
when acceleration >= 0 and acceleration < 7 then 'Fast Car'
when acceleration >= 7 and acceleration < 12 then 'Average Car'
when acceleration >= 12  then 'Slow Car' 
end as Car_Type,acceleration from  answer_2mpg order by Car_Type asc
""").show(10)

+-----------+------------+
|   Car_Type|acceleration|
+-----------+------------+
|Average Car|         8.0|
|Average Car|        11.0|
|Average Car|         9.5|
|Average Car|        11.5|
|Average Car|        10.0|
|Average Car|        10.5|
|Average Car|        11.5|
|Average Car|        10.0|
|Average Car|        11.5|
|Average Car|         8.5|
+-----------+------------+
only showing top 10 rows



There is no fast car in this dataset accoring to given flag

Question 3: We want to predict mpg for given automobile info. (40 points)
Choose one of the ML algorithms from Spark ML library and prepare data for
training

In [179]:
spark = SparkSession.builder.appName("Spark Dataframe Clustering").getOrCreate()
mpgquestion3 = spark.read\
          .option("header","true")\
          .option("inferSchema", "true")\
          .option("delimiter",",")\
          .csv("auto-mpg.data.txt")

In [199]:
mpgquestion3.show(4)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 4 rows



In [181]:
mpgquestion3.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model_year: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- name: string (nullable = true)



I wanted to check whether any null values in our data or not. According to result below there is no null values in our dataset

In [182]:
mpgquestion3.createOrReplaceTempView("answer_3mpg")

res_answer_one_df = spark.sql("""
select count(*), 'null_mpg' as flag from answer_3mpg where mpg is  null
union all
select count(*), 'null_cylinders' as flag from answer_3mpg where cylinders is null
union all
select count(*), 'null_displacement' as flag from answer_3mpg where displacement is null
union all
select count(*), 'null_horsepower' as flag from answer_3mpg where horsepower is null
union all
select count(*), 'null_weight' as flag from answer_3mpg where weight is null
union all
select count(*), 'null_acceleration' as flag from answer_3mpg where acceleration is null
union all
select count(*), 'null_model_year' as flag from answer_3mpg where model_year is null
union all
select count(*), 'null_origin' as flag from answer_3mpg where origin is null
union all
select count(*), 'null_name' as flag from answer_3mpg where name is null
""").show()

+--------+-----------------+
|count(1)|             flag|
+--------+-----------------+
|       0|         null_mpg|
|       0|   null_cylinders|
|       0|null_displacement|
|       0|  null_horsepower|
|       0|      null_weight|
|       0|null_acceleration|
|       0|  null_model_year|
|       0|      null_origin|
|       0|        null_name|
+--------+-----------------+



In [213]:
#But horse power has ? we need to solve this poblem
mpgquestion3 = spark.sql("""select  
mpg,cylinders,displacement,weight,horsepower,acceleration,model_year,origin,name
from answer_3mpg where horsepower <> '?'""")

In [ ]:
#I prefer to delefe ? from the horsepower

In [214]:
mpgquestion3.show()

+----+---------+------------+------+----------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|weight|horsepower|acceleration|model_year|origin|                name|
+----+---------+------------+------+----------+------------+----------+------+--------------------+
|18.0|        8|       307.0|  3504|       130|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|  3693|       165|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|  3436|       150|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|  3433|       150|        12.0|        70|     1|       amc rebel sst|
|17.0|        8|       302.0|  3449|       140|        10.5|        70|     1|         ford torino|
|15.0|        8|       429.0|  4341|       198|        10.0|        70|     1|    ford galaxie 500|
|14.0|        8|       454.0|  4354|       220|         9.0|        70|     1|    chevrolet impala|


In [ ]:
#Makind dependent variable mpg indexing.

In [215]:
indxr = StringIndexer(inputCol="mpg",outputCol="label")
indexModel = indxr.fit(mpgquestion3)
mpgquestion3indexed = indexModel.transform(mpgquestion3)

In [216]:
mpgquestion3indexed.show(3)

+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+
| mpg|cylinders|displacement|weight|horsepower|acceleration|model_year|origin|                name|label|
+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+
|18.0|        8|       307.0|  3504|       130|        12.0|        70|     1|chevrolet chevell...|  2.0|
|15.0|        8|       350.0|  3693|       165|        11.5|        70|     1|   buick skylark 320|  3.0|
|18.0|        8|       318.0|  3436|       150|        11.0|        70|     1|  plymouth satellite|  2.0|
+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+
only showing top 3 rows



One-Hot Encoding origin

In [217]:
encoder = OneHotEncoder(inputCol="origin",outputCol="encoded_Origin")


In [ ]:
#One-hot encoder is a encoder that is used for preparing categorical data to ml algorithms. In case of distinct values of the field is higher than 2 we can use one hot encoder, if it is equal to 2 we can use label encoder.

In [218]:
mpgquestion3encoded = encoder.fit(mpgquestion3indexed).transform(mpgquestion3indexed)
mpgquestion3encoded.show(3)

+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+--------------+
| mpg|cylinders|displacement|weight|horsepower|acceleration|model_year|origin|                name|label|encoded_Origin|
+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+--------------+
|18.0|        8|       307.0|  3504|       130|        12.0|        70|     1|chevrolet chevell...|  2.0| (3,[1],[1.0])|
|15.0|        8|       350.0|  3693|       165|        11.5|        70|     1|   buick skylark 320|  3.0| (3,[1],[1.0])|
|18.0|        8|       318.0|  3436|       150|        11.0|        70|     1|  plymouth satellite|  2.0| (3,[1],[1.0])|
+----+---------+------------+------+----------+------------+----------+------+--------------------+-----+--------------+
only showing top 3 rows



In [219]:
 mpgquestion3encoded = mpgquestion3encoded.select("label","cylinders","displacement","horsepower","weight","acceleration","model_year",
 "encoded_Origin") # dropping normal origin continue with encoded origin.Also name cannot be used for Ml model

In [220]:
mpgquestion3encoded.createOrReplaceTempView("string_to_int_horsepower")

In [171]:
#IllegalArgumentException: Data type string of column horsepower is not supported. Below step is made to solve this problem

In [221]:
mpgquestion3encoded = spark.sql("""select 
label,cast(cylinders as int) as cylinders,
cast(displacement as int) as displacement,
cast(horsepower as int) as horsepower,
cast(weight as int),
cast(acceleration as int),
cast(model_year as int),
encoded_Origin
from string_to_int_horsepower
 """) 

In [222]:
mpgquestion3encoded.show(3)

+-----+---------+------------+----------+------+------------+----------+--------------+
|label|cylinders|displacement|horsepower|weight|acceleration|model_year|encoded_Origin|
+-----+---------+------------+----------+------+------------+----------+--------------+
|  2.0|        8|         307|       130|  3504|          12|        70| (3,[1],[1.0])|
|  3.0|        8|         350|       165|  3693|          11|        70| (3,[1],[1.0])|
|  2.0|        8|         318|       150|  3436|          11|        70| (3,[1],[1.0])|
+-----+---------+------------+----------+------+------------+----------+--------------+
only showing top 3 rows



In [223]:
mpgquestion3encoded.printSchema()

root
 |-- label: double (nullable = false)
 |-- cylinders: integer (nullable = true)
 |-- displacement: integer (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- acceleration: integer (nullable = true)
 |-- model_year: integer (nullable = true)
 |-- encoded_Origin: vector (nullable = true)



In [224]:
vec = VectorAssembler(inputCols=mpgquestion3encoded.columns[1:7],outputCol="features")

In [225]:
mpgquestion3encoded = vec.transform(mpgquestion3encoded)
mpgquestion3encoded = mpgquestion3encoded.select("features","label")

In [226]:
mpgquestion3encoded.show(truncate=False)

+----------------------------------+-----+
|features                          |label|
+----------------------------------+-----+
|[8.0,307.0,130.0,3504.0,12.0,70.0]|2.0  |
|[8.0,350.0,165.0,3693.0,11.0,70.0]|3.0  |
|[8.0,318.0,150.0,3436.0,11.0,70.0]|2.0  |
|[8.0,304.0,150.0,3433.0,12.0,70.0]|5.0  |
|[8.0,302.0,140.0,3449.0,10.0,70.0]|15.0 |
|[8.0,429.0,198.0,4341.0,10.0,70.0]|3.0  |
|[8.0,454.0,220.0,4354.0,9.0,70.0] |1.0  |
|[8.0,440.0,215.0,4312.0,8.0,70.0] |1.0  |
|[8.0,455.0,225.0,4425.0,10.0,70.0]|1.0  |
|[8.0,390.0,190.0,3850.0,8.0,70.0] |3.0  |
|[8.0,383.0,170.0,3563.0,10.0,70.0]|3.0  |
|[8.0,340.0,160.0,3609.0,8.0,70.0] |1.0  |
|[8.0,400.0,150.0,3761.0,9.0,70.0] |3.0  |
|[8.0,455.0,225.0,3086.0,10.0,70.0]|1.0  |
|[4.0,113.0,95.0,2372.0,15.0,70.0] |7.0  |
|[6.0,198.0,95.0,2833.0,15.0,70.0] |8.0  |
|[6.0,199.0,97.0,2774.0,15.0,70.0] |2.0  |
|[6.0,200.0,85.0,2587.0,16.0,70.0] |16.0 |
|[4.0,97.0,88.0,2130.0,14.0,70.0]  |13.0 |
|[4.0,97.0,46.0,1835.0,20.0,70.0]  |4.0  |
+----------

In [227]:
rfClassifier = RandomForestClassifier()

In [228]:
trainDF, testDF = mpgquestion3encoded.randomSplit([0.7,0.3],seed=10)

In [229]:
eva = MulticlassClassificationEvaluator(metricName="accuracy")

In [245]:
params = ParamGridBuilder()\
    .addGrid(rfClassifier.maxDepth, [17,18,19]) \
    .addGrid(rfClassifier.numTrees, [2,3,4,5])\
    .addGrid(rfClassifier.impurity, ['gini',"entropy"])\
    .build()

validator = TrainValidationSplit(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                trainRatio=0.8
                                )


In [246]:
model = validator.fit(mpgquestion3encoded)

print("Num Trees : ",model.bestModel.getNumTrees)
print("Max Depth : ",model.bestModel._java_obj.getMaxDepth())
print("Impurtiy : ",model.bestModel._java_obj.getImpurity())

sonucDF = model.transform(testDF)

basari = eva.evaluate(sonucDF)

print("Başarı : ",basari)

Num Trees :  5
Max Depth :  17
Impurtiy :  entropy
Başarı :  0.864406779661017


In [ ]:
#After increasing de maxdepth from 1,2,3 to 13,14,15 our accuracy increased dramatically from 0.23 to 0.86

In [ ]:
#Trying to pca for model

In [275]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

pca_prep = mpgquestion3encoded.select("features","label")
pca = PCA(k=3, inputCol="features", outputCol="Features")
model = pca.fit(pca_prep)
result = model.transform(pca_prep).select("features","label")
result.show(truncate=False)


+------------------------------------------------------------+-----+
|features                                                    |label|
+------------------------------------------------------------+-----+
|[-3518.4019375612384,95.94136835581713,-17.62639983600673]  |2.0  |
|[-3712.2991077500105,67.62104611725773,-37.40704091055741]  |3.0  |
|[-3452.9385526808746,71.32756736040149,-33.511778972374586] |2.0  |
|[-3448.3584338019305,84.24021666343378,-37.68554887992415]  |5.0  |
|[-3463.6254528129457,90.98036253179399,-28.92156754827797]  |15.0 |
|[-4365.871804778685,61.27380056117486,-43.24373303518897]   |3.0  |
|[-4382.493703537521,32.602895728113985,-56.58225637193008]  |1.0  |
|[-4339.007083709525,42.23116338533601,-56.26494170804667]   |1.0  |
|[-4453.281397902522,38.77255139297322,-60.76942706648724]   |1.0  |
|[-3873.7006277784067,41.190424924538156,-48.8452547602388]  |3.0  |
|[-3587.221003339059,19.185515238073695,-32.544419521580004] |3.0  |
|[-3627.5801728202073,68.328027698

In [276]:
rfClassifier = RandomForestClassifier()
trainDF, testDF = result.randomSplit([0.7,0.3],seed=10)
eva = MulticlassClassificationEvaluator(metricName="accuracy")

In [281]:
params = ParamGridBuilder()\
    .addGrid(rfClassifier.maxDepth, [17,18,19]) \
    .addGrid(rfClassifier.numTrees, [13,14,15])\
    .addGrid(rfClassifier.impurity, ['gini',"entropy"])\
    .build()

validator = TrainValidationSplit(
                                estimatorParamMaps=params,
                                estimator=rfClassifier,
                                evaluator=eva,
                                trainRatio=0.8
                                )


In [282]:
model = validator.fit(result)

print("Num Trees : ",model.bestModel.getNumTrees)
print("Max Depth : ",model.bestModel._java_obj.getMaxDepth())
print("Impurtiy : ",model.bestModel._java_obj.getImpurity())

sonucDF = model.transform(testDF)

basari = eva.evaluate(sonucDF)

print("Başarı : ",basari)

Num Trees :  13
Max Depth :  17
Impurtiy :  entropy
Başarı :  0.940677966101695


In [ ]:
#we can see that after using pca and increasing the numTrees our accuracy increased from 86 to 94. We can use pca for this data set with the given parameters below.